# 03 – Tissue Classification & 3D Spatial Inference

This notebook performs:
1. **Tissue subclassification** – Separate parenchyma, connective tissue, ducts/vessels, and background
2. **Lumen detection** – Identify hollow structures (ducts, vessels)
3. **3D spatial inference** – Estimate anatomical position (head/body/tail, depth) based on tissue features
4. **Comprehensive visualizations** – Full-image and close-up tissue maps

Handles brightfield images with variable staining intensity.

In [ ]:
import os
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

os.environ.setdefault('OPENCV_IO_MAX_IMAGE_PIXELS', str(2**63 - 1))

import cv2
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, Patch
from matplotlib.colors import ListedColormap
from skimage.color import label2rgb
from skimage.segmentation import find_boundaries
import pandas as pd

from isletscope.tissue_classification import TissueClassifier
from isletscope.spatial_inference import SpatialInferer

print('Imports complete')

## Configuration

In [ ]:
# ===== Input =====
output_dir = Path('../outputs')
output_dir.mkdir(exist_ok=True)

# Load results from previous notebooks
img_normalized_path = output_dir / 'img_normalized.npy'
islet_labels_path = output_dir / 'islet_labels.npy'
cell_mask_path = output_dir / 'cell_mask.npy'

# ===== Tissue Classification =====
n_clusters = 4  # K-means clusters for initial tissue classification
luminal_threshold = 220  # Grayscale threshold for lumen detection
min_lumen_area = 64  # Minimum pixels for lumen structure

# ===== 3D Inference =====
known_location = None  # If known, set to 'head', 'body', or 'tail'
known_depth = None  # If known, set to float 0-1 (0=surface, 1=deep)

# ===== Visualization =====
closeup_regions = [
    (0.2, 0.4, 0.3, 0.5),  # Region 1
    (0.5, 0.7, 0.6, 0.8),  # Region 2
]

# Tissue class colors for visualization
tissue_colors = {
    'parenchyma': [200, 100, 100],      # Reddish
    'connective': [100, 200, 100],      # Greenish
    'ducts_vessels': [100, 100, 200],   # Bluish
    'background': [50, 50, 50],         # Dark gray
    'other': [150, 150, 150]            # Light gray
}

print('Configuration set')

## Load Data

In [ ]:
# Load required arrays
img_normalized = np.load(img_normalized_path)
islet_labels = np.load(islet_labels_path) if islet_labels_path.exists() else None
cell_mask = np.load(cell_mask_path) if cell_mask_path.exists() else None

print(f'Image shape: {img_normalized.shape}')
if islet_labels is not None:
    print(f'Islets: {islet_labels.max()}')
if cell_mask is not None:
    print(f'Cell coverage: {100*cell_mask.sum()/cell_mask.size:.1f}%')

def get_closeup_coords(img_shape, region_frac):
    h, w = img_shape[:2]
    y1, y2, x1, x2 = region_frac
    return (int(y1*h), int(y2*h), int(x1*w), int(x2*w))

## Tissue Classification

Classify pixels into parenchyma, connective tissue, ducts/vessels, background, and other.

In [ ]:
print('Running tissue classification...')
classifier = TissueClassifier(
    n_clusters=n_clusters,
    luminal_threshold=luminal_threshold,
    min_lumen_area=min_lumen_area
)

tissue_masks = classifier.classify(img_normalized)

print('\nTissue class pixel counts:')
for tissue_type, mask in tissue_masks.items():
    count = int(mask.sum())
    pct = 100 * count / mask.size
    print(f'  {tissue_type:15s}: {count:8d} pixels ({pct:5.1f}%)')

In [ ]:
# Create composite tissue map
tissue_map = np.zeros(img_normalized.shape[:2], dtype=np.uint8)
tissue_legend = {}
label_idx = 1

for tissue_type in ['parenchyma', 'connective', 'ducts_vessels', 'background', 'other']:
    if tissue_type in tissue_masks:
        tissue_map[tissue_masks[tissue_type] > 0] = label_idx
        tissue_legend[label_idx] = tissue_type
        label_idx += 1

# Full image visualization
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# Original image
axes[0, 0].imshow(cv2.cvtColor(img_normalized, cv2.COLOR_BGR2RGB))
axes[0, 0].set_title('Normalized Image', fontsize=14, fontweight='bold')
axes[0, 0].axis('off')

# Individual tissue masks
mask_titles = ['parenchyma', 'connective', 'ducts_vessels', 'background', 'other']
for idx, tissue_type in enumerate(mask_titles[:5]):
    row = (idx + 1) // 3
    col = (idx + 1) % 3
    if tissue_type in tissue_masks:
        axes[row, col].imshow(tissue_masks[tissue_type], cmap='gray')
        axes[row, col].set_title(tissue_type.replace('_', ' ').title(), fontsize=12, fontweight='bold')
    else:
        axes[row, col].text(0.5, 0.5, 'N/A', ha='center', va='center', fontsize=16)
        axes[row, col].set_title(tissue_type.replace('_', ' ').title(), fontsize=12)
    axes[row, col].axis('off')

plt.tight_layout()
plt.savefig(output_dir / '05_tissue_individual_masks.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Composite tissue map with color coding
tissue_rgb = np.zeros((*img_normalized.shape[:2], 3), dtype=np.uint8)

for tissue_type, mask in tissue_masks.items():
    if tissue_type in tissue_colors:
        color = tissue_colors[tissue_type]
        tissue_rgb[mask > 0] = color

# Create overlay
overlay = cv2.cvtColor(img_normalized, cv2.COLOR_BGR2RGB).copy().astype(float)
overlay = overlay * 0.4 + tissue_rgb.astype(float) * 0.6
overlay = overlay.astype(np.uint8)

fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# Original
axes[0].imshow(cv2.cvtColor(img_normalized, cv2.COLOR_BGR2RGB))
axes[0].set_title('Original Image', fontsize=14, fontweight='bold')
axes[0].axis('off')

# Tissue map
axes[1].imshow(tissue_rgb)
axes[1].set_title('Tissue Classification Map', fontsize=14, fontweight='bold')
axes[1].axis('off')

# Add legend
legend_elements = [Patch(facecolor=np.array(tissue_colors[k])/255, label=k.replace('_', ' ').title())
                   for k in tissue_colors.keys() if k in tissue_masks]
axes[1].legend(handles=legend_elements, loc='upper right', fontsize=10)

# Overlay
axes[2].imshow(overlay)
axes[2].set_title('Overlay', fontsize=14, fontweight='bold')
axes[2].axis('off')

# Mark closeup regions
for i, region_frac in enumerate(closeup_regions):
    y1, y2, x1, x2 = get_closeup_coords(img_normalized.shape, region_frac)
    rect = Rectangle((x1, y1), x2-x1, y2-y1, linewidth=2, edgecolor='white', facecolor='none')
    axes[2].add_patch(rect)
    axes[2].text(x1, y1-10, f'Region {i+1}', color='white', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.savefig(output_dir / '05_tissue_classification_full.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Closeup views
n_regions = len(closeup_regions)
fig, axes = plt.subplots(n_regions, 3, figsize=(15, 5*n_regions))
if n_regions == 1:
    axes = axes.reshape(1, -1)

for i, region_frac in enumerate(closeup_regions):
    y1, y2, x1, x2 = get_closeup_coords(img_normalized.shape, region_frac)
    
    img_crop = img_normalized[y1:y2, x1:x2]
    tissue_crop = tissue_rgb[y1:y2, x1:x2]
    overlay_crop = overlay[y1:y2, x1:x2]
    
    # Image
    axes[i, 0].imshow(cv2.cvtColor(img_crop, cv2.COLOR_BGR2RGB))
    axes[i, 0].set_title(f'Region {i+1} - Image', fontsize=12, fontweight='bold')
    axes[i, 0].axis('off')
    
    # Tissue map
    axes[i, 1].imshow(tissue_crop)
    axes[i, 1].set_title(f'Region {i+1} - Tissue Map', fontsize=12, fontweight='bold')
    axes[i, 1].axis('off')
    
    # Overlay
    axes[i, 2].imshow(overlay_crop)
    axes[i, 2].set_title(f'Region {i+1} - Overlay', fontsize=12, fontweight='bold')
    axes[i, 2].axis('off')

plt.tight_layout()
plt.savefig(output_dir / '05_tissue_classification_closeup.png', dpi=150, bbox_inches='tight')
plt.show()

## 3D Spatial Inference

Estimate anatomical position based on tissue features, vessel density, and islet distribution.

In [ ]:
print('Running 3D spatial inference...')
inferer = SpatialInferer()

# Prepare inputs
vessel_mask = tissue_masks.get('ducts_vessels')
islet_mask = (islet_labels > 0).astype(np.uint8) if islet_labels is not None else None

coords = inferer.infer(
    img_normalized,
    vessel_mask=vessel_mask,
    islet_mask=islet_mask,
    tissue_masks=tissue_masks
)

print('\n=== Estimated 3D Coordinates ===')
print(f"Anatomical location: {coords['location']}")
print(f"  Confidence: {coords['location_confidence']:.2f}")
print(f"  Position score: {coords['position_score']:.2f} (0=head, 0.5=body, 1=tail)")
print(f"\nEstimated depth: {coords['depth']:.2f}")
print(f"  Confidence: {coords['depth_confidence']:.2f}")
print(f"  (0=surface, 1=deep)")

if 'features' in coords:
    print('\n=== Features Used ===')
    for key, val in coords['features'].items():
        if isinstance(val, float):
            print(f"  {key}: {val:.3f}")
        else:
            print(f"  {key}: {val}")

In [ ]:
# Visualize spatial features
fig, axes = plt.subplots(2, 2, figsize=(14, 12))

# Original with annotation
axes[0, 0].imshow(cv2.cvtColor(img_normalized, cv2.COLOR_BGR2RGB))
title_text = f"Estimated Location: {coords['location'].upper()}"
axes[0, 0].set_title(title_text, fontsize=14, fontweight='bold')
axes[0, 0].text(10, 30, f"Confidence: {coords['location_confidence']:.2%}", 
                bbox=dict(boxstyle='round', facecolor='white', alpha=0.8),
                fontsize=12, fontweight='bold')
axes[0, 0].axis('off')

# Vessel density map
if vessel_mask is not None:
    from scipy.ndimage import gaussian_filter
    vessel_density = gaussian_filter(vessel_mask.astype(float), sigma=20)
    im1 = axes[0, 1].imshow(vessel_density, cmap='hot')
    axes[0, 1].set_title('Vessel Density (smoothed)', fontsize=12, fontweight='bold')
    axes[0, 1].axis('off')
    plt.colorbar(im1, ax=axes[0, 1], fraction=0.046)
else:
    axes[0, 1].text(0.5, 0.5, 'No vessel mask', ha='center', va='center', fontsize=14)
    axes[0, 1].axis('off')

# Islet distribution
if islet_mask is not None:
    from scipy.ndimage import gaussian_filter
    islet_density = gaussian_filter(islet_mask.astype(float), sigma=20)
    im2 = axes[1, 0].imshow(islet_density, cmap='viridis')
    axes[1, 0].set_title('Islet Density (smoothed)', fontsize=12, fontweight='bold')
    axes[1, 0].axis('off')
    plt.colorbar(im2, ax=axes[1, 0], fraction=0.046)
else:
    axes[1, 0].text(0.5, 0.5, 'No islet mask', ha='center', va='center', fontsize=14)
    axes[1, 0].axis('off')

# 3D position diagram
axes[1, 1].set_xlim(0, 1)
axes[1, 1].set_ylim(0, 1)
axes[1, 1].set_aspect('equal')

# Draw pancreas schematic (head-body-tail)
from matplotlib.patches import Ellipse, FancyBboxPatch
axes[1, 1].add_patch(FancyBboxPatch((0.05, 0.4), 0.9, 0.2, 
                                     boxstyle="round,pad=0.05", 
                                     edgecolor='black', facecolor='lightgray', linewidth=2))
axes[1, 1].text(0.15, 0.5, 'HEAD', ha='center', va='center', fontsize=12, fontweight='bold')
axes[1, 1].text(0.5, 0.5, 'BODY', ha='center', va='center', fontsize=12, fontweight='bold')
axes[1, 1].text(0.85, 0.5, 'TAIL', ha='center', va='center', fontsize=12, fontweight='bold')

# Mark estimated position
pos_x = coords['position_score']
axes[1, 1].plot(pos_x, 0.5, 'ro', markersize=20, label='Estimated position')
axes[1, 1].arrow(pos_x, 0.7, 0, -0.15, head_width=0.03, head_length=0.03, fc='red', ec='red')

# Depth indicator
depth_val = coords['depth']
axes[1, 1].text(0.5, 0.2, f"Depth: {depth_val:.2f}\n(0=surface, 1=deep)", 
                ha='center', va='center', fontsize=11,
                bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.5))

axes[1, 1].set_title('Estimated 3D Position in Pancreas', fontsize=12, fontweight='bold')
axes[1, 1].axis('off')

plt.tight_layout()
plt.savefig(output_dir / '06_spatial_inference.png', dpi=150, bbox_inches='tight')
plt.show()

## Analysis Summary

In [ ]:
# Create comprehensive summary
summary = {
    'Image': {
        'Shape': f"{img_normalized.shape[1]} x {img_normalized.shape[0]} px",
        'Area': f"{img_normalized.shape[0] * img_normalized.shape[1]:,} px²"
    },
    'Cells': {},
    'Islets': {},
    'Tissue Classification': {},
    'Spatial Position': {
        'Location': f"{coords['location']} (confidence: {coords['location_confidence']:.1%})",
        'Depth': f"{coords['depth']:.2f} (confidence: {coords['depth_confidence']:.1%})"
    }
}

if cell_mask is not None:
    summary['Cells'] = {
        'Total area': f"{cell_mask.sum():,} px",
        'Coverage': f"{100*cell_mask.sum()/cell_mask.size:.1f}%"
    }

if islet_labels is not None:
    summary['Islets'] = {
        'Count': int(islet_labels.max()),
        'Total area': f"{(islet_labels > 0).sum():,} px",
        'Coverage': f"{100*(islet_labels > 0).sum()/islet_labels.size:.2f}%"
    }

for tissue_type, mask in tissue_masks.items():
    summary['Tissue Classification'][tissue_type] = f"{100*mask.sum()/mask.size:.1f}%"

print('\n' + '='*60)
print('ANALYSIS SUMMARY'.center(60))
print('='*60)

for section, data in summary.items():
    print(f'\n{section}:')
    if isinstance(data, dict):
        for key, val in data.items():
            print(f'  {key:20s}: {val}')
    else:
        print(f'  {data}')

print('\n' + '='*60)

## Save Results

In [ ]:
# Save tissue masks
for tissue_type, mask in tissue_masks.items():
    np.save(output_dir / f'tissue_{tissue_type}.npy', mask)

# Save composite tissue map
np.save(output_dir / 'tissue_map.npy', tissue_map)
np.save(output_dir / 'tissue_rgb.npy', tissue_rgb)

# Save spatial coordinates as JSON
import json
# Convert numpy types to native Python types
coords_serializable = {}
for key, val in coords.items():
    if isinstance(val, np.ndarray):
        coords_serializable[key] = val.tolist()
    elif isinstance(val, (np.integer, np.floating)):
        coords_serializable[key] = float(val)
    elif isinstance(val, dict):
        coords_serializable[key] = {k: float(v) if isinstance(v, (np.integer, np.floating)) else v 
                                     for k, v in val.items()}
    else:
        coords_serializable[key] = val

with open(output_dir / 'spatial_coordinates.json', 'w') as f:
    json.dump(coords_serializable, f, indent=2)

# Save summary as CSV
summary_flat = []
for section, data in summary.items():
    if isinstance(data, dict):
        for key, val in data.items():
            summary_flat.append({'Section': section, 'Metric': key, 'Value': val})
    else:
        summary_flat.append({'Section': section, 'Metric': '', 'Value': data})

summary_df = pd.DataFrame(summary_flat)
summary_df.to_csv(output_dir / 'analysis_summary.csv', index=False)

print(f'Results saved to {output_dir}')
print(f'\nKey outputs:')
print(f'  - Tissue masks: tissue_*.npy')
print(f'  - Spatial coordinates: spatial_coordinates.json')
print(f'  - Analysis summary: analysis_summary.csv')
print(f'\n✓ Analysis pipeline complete!')